In [30]:
import pandas as pd
import re

doc = []
with open('assets/dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0                        03/25/93 Total time of visit (in minutes):\n
1                                      6/18/85 Primary Care Doctor:\n
2            sshe plans to move as of 7/8/71 In-Home Services: None\n
3                               7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (Numeric Scale): 7\n
5                                   .Per 7/06/79 Movement D/O note:\n
6      4, 5/18/78 Patient's thoughts about current substance abuse:\n
7        10/24/89 CPT Code: 90801 - Psychiatric Diagnosis Interview\n
8                                        3/7/86 SOS-10 Total Score:\n
9                            (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [152]:
def date_sorter():
   
    order = None
    df_lista=df.copy().tolist()
    formatos_fecha = {
            1:r'((?<!\.)(\d{1,2})[/-](\d{1,2})[/-](\d{2,4}))', #cada patron debe estar entre parentesis si se desea extraer y dentro del mismo cada porcion que queremos mostrar como campo independiente tambien entre parentesis
            2:r'(((?:\d{1,2} ))?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z]*[?:, -]* (\d{4}))',
            3:r'(((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z]*[-.]* (\d{1,2})(?:st|nd|rd|th)*,? (\d{2,4}))',
            4:r'((\d{1,2})/(\d{4}))',
            5:r'([1-2]\d{3})',
    }
    mes_num=['01','02','03','04','05','06','07','08','09','10','11','12']
    mes_abr=['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec']
    #mes=['january','february','march','april','may','june','july','august','september','october','november','december']

    extracciones=[]
    #indices=[]
    for i in formatos_fecha.keys():
        nombre_extraccion=f'extraccion_{i}'#generamos nombre de variable
        nombre_extraccion= df.str.extractall(formatos_fecha[i])#la rellenamos con un dataframe
        nombre_extraccion.reset_index(inplace=True)#al resetear los indices del dataframe (representado por la variable previa) se genera una columna con los indices basales 'level_0'
        extracciones.append(nombre_extraccion)#guardamos la variable RELLENADA en una lista fuera del bucle para que persista 
        #indices_extraccion=f'indices_extraccion_{i}'
        #indices_extraccion = nombre_extraccion['level_0']
        #indices.append(indices_extraccion)
    
    #nombres de las columnas y edicion
    extracciones[0].columns=['level_0','match','Fecha','Mes','Dia','Año']#cambiar nombres de columnas existentes
    extracciones[0]['Año']=extracciones[0]['Año'].apply(str)#transformar en string los elementos de una columna para poder tratarlos como tales
    extracciones[0]['Año']=extracciones[0]['Año'].apply(lambda x: '19'+x if len(x)<=2 else x)#añadir '19' si se cumple la condicion de que la longitud del valor es <2
    extracciones[0]['Dia']=extracciones[0]['Dia'].apply(str)
    extracciones[0]['Dia']=extracciones[0]['Dia'].apply(lambda x: '0'+x if len(x)<2 else x)
    extracciones[0]['Mes']=extracciones[0]['Mes'].apply(str)
    extracciones[0]['Mes']=extracciones[0]['Mes'].apply(lambda x: '0'+x if len(x)<2 else x)
    extracciones[0]['Nota']=extracciones[0]['level_0'].apply(lambda x: df_lista[x])#añadimos AL FINAL una nueva columna 'Nota' con los valores de df_lista segun su posicion contenida en otra variable

    extracciones[1].columns=['level_0','match','Fecha','Dia','Mes','Año']
    extracciones[1]['Mes']=extracciones[1]['Mes'].apply(lambda x: mes_num[mes_abr.index(x.lower().strip())])
    extracciones[1]['Dia']=extracciones[1]['Dia'].apply(str)
    extracciones[1]['Dia']=extracciones[1]['Dia'].apply(lambda x: x.replace('nan','01'))
    extracciones[1]['Nota']=extracciones[1]['level_0'].apply(lambda x: df_lista[x])
    
    extracciones[2].columns=['level_0','match','Fecha','Mes','Dia','Año']
    extracciones[2]['Mes']=extracciones[2]['Mes'].apply(lambda x: mes_num[mes_abr.index(x.lower())])#cambiamos un valor por otro a partir de su posicion (la comparten en dos listas distintas)
    extracciones[2]['Nota']=extracciones[2]['level_0'].apply(lambda x: df_lista[x])

    extracciones[3].columns=['level_0','match','Fecha','Mes','Año']
    extracciones[3].insert(4,'Dia','01')
    extracciones[3]['Mes']=extracciones[3]['Mes'].apply(lambda x: '0'+x if len(x)<2 else x)
    extracciones[3]['Nota']=extracciones[3]['level_0'].apply(lambda x: df_lista[x])

    extracciones[4].columns=['level_0','match','Fecha']
    extracciones[4].insert(3,'Mes','01')
    extracciones[4].insert(4,'Dia','01')
    extracciones[4]['Año']=extracciones[4]['Fecha']
    extracciones[4]['Nota']=extracciones[4]['level_0'].apply(lambda x: df_lista[x])
    
    #seleccion de extracciones de mayor cantidad de informacion
    extracciones_0_1_2 = pd.concat([extracciones[0],extracciones[1],extracciones[2]]).sort_values(by=['level_0'], ascending=[True])
    extracciones_0_1_2.reset_index(inplace=True)#al resetear indice para unificar el df concatenado se crea una columna 'index' porque 'level_0' ya existe
    extracciones_0_1_2.drop('index',axis=1,inplace=True) #la eliminamos (ya tenemos level_0)
    #indices_juntos=extracciones_juntas['level_0']
    
    #eliminar redundantes en extracciones[3] para concatenarlo a los anteriores
    #'level_0' es la columna clave que identifica las filas redundantes ya que es compartida por todos los df
    columna_clave = 'level_0'
    # Encuentra las filas en extracciones[3] que están en extracciones_0_1_2
    filas_comunes = extracciones[3][extracciones[3][columna_clave].isin(extracciones_0_1_2[columna_clave])]
    # Utiliza drop para eliminar las filas comunes 
    extracciones[3] = extracciones[3].drop(filas_comunes.index)
    #concatenamos el resultado a lo previo
    extracciones_0_1_2_3 = pd.concat([extracciones_0_1_2,extracciones[3]]).sort_values(by=['level_0'], ascending=[True])
    extracciones_0_1_2_3.reset_index(inplace=True)
    extracciones_0_1_2_3.drop('index',axis=1,inplace=True) 
    #Repetimos el proceso con extracciones[4]
    filas_comunes = extracciones[4][extracciones[4][columna_clave].isin(extracciones_0_1_2_3[columna_clave])]
    extracciones[4] = extracciones[4].drop(filas_comunes.index)
    extracciones_0_1_2_3_4 = pd.concat([extracciones_0_1_2_3,extracciones[4]]).sort_values(by=['level_0'], ascending=[True])
    extracciones_0_1_2_3_4.reset_index(inplace=True)
    extracciones_0_1_2_3_4.drop('index',axis=1,inplace=True)

    #ordenar el df por el valor fecha y para los empatados usar el indice inicial (level_0)
    #asegurar que no haya errores de formato
    campos=['Dia','Mes','Año']
    for campo in campos:
        extracciones_0_1_2_3_4[campo]=extracciones_0_1_2_3_4[campo].apply(str)
        extracciones_0_1_2_3_4[campo]=extracciones_0_1_2_3_4[campo].apply(lambda x: x.replace(x,x.strip()))
    
    #crear un campo datetime para poder ordenar por fecha
    extracciones_0_1_2_3_4['Fecha_DT'] =pd.to_datetime(extracciones_0_1_2_3_4['Mes']+'/'+extracciones_0_1_2_3_4['Dia']+'/'+extracciones_0_1_2_3_4['Año'])
    #ordenar por fecha e indice
    extracciones_order=extracciones_0_1_2_3_4.sort_values(by=['Fecha_DT', 'level_0'], ascending=[True, True])
    #fijar el indice
    extracciones_order=extracciones_order.explode('level_0').reset_index(drop=True)
    
    #raise NotImplementedError()
    #generar el objeto Series
    order = pd.Series(extracciones_order['level_0'])

    return order

In [153]:
pd.set_option('display.max_rows', None)  # Mostrar todas las filas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)#muestra todo el ancho de columna
# Restablece las opciones de visualización a su valor predeterminado
#pd.reset_option('display.max_rows')
#pd.reset_option('display.max_columns')
date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
30     162
31     154
32     402
33      95
34      73
35     108
36     156
37     332
38     182
39      82
40     351
41     278
42     214
43     155
44     223
45     473
46      49
47     317
48      11
49     319
50      40
51     418
52     165
53     370
54     382
55       3
56      50
57     363
58     219
59     465
60     237
61      23
62     342
63     204
64     258
65     315
66      27
67      93
68      17
69     303
70     488
71     283
72     395
73     309
74     419
75     123
76      19
77     117
78     232
79      72
80     189
81     318
82     369
83     493
84     239
85     148
86     105
87     336
88       6
89     200
90      81